In [5]:
import numpy as np

In [6]:
# Parameters defined in paper
N = 139 # Matrix and vector size
s = 0  # Magic number
p = 9223372036853751941 # Modulus

In [8]:
def generate_mixmax_matrix():
    
    # Initialize the MIXMAX matrix with zeros
    matrix = np.ones((N, N), 1, dtype=int)
    
    # Fill the matrix with the specified pattern
    for i in range(N):
        for j in range(N):
            if i == j and j > 0:
                matrix[i][j] = 2
            elif j < i and j > 0:
                matrix[i][j] = (i + 1) - (j - 1)
    # Add magic number
    matrix[2][1] += s

    return matrix

mixmax_matrix = generate_mixmax_matrix()

print(mixmax_matrix)

[[  1   1   1 ...   1   1   1]
 [  1   2   1 ...   1   1   1]
 [  1   3   2 ...   1   1   1]
 ...
 [  1 137 136 ...   2   1   1]
 [  1 138 137 ...   3   2   1]
 [  1 139 138 ...   4   3   2]]


In [9]:
def generate_initial_vector(seed):

    # Pseudo random vector from seed
    a = 6364136223846793005
    c = 1

    random_numbers = []
    x = seed
    for _ in range(N):
        x = ((a * x + c) % p) / p
        random_numbers.append(x)
    return random_numbers

initial_vector = generate_initial_vector(42)

print(initial_vector)

[0.9800433435344447, 0.6762309184326339, 0.46660003158132946, 0.32195450331717784, 0.2221489395423314, 0.15328299753945182, 0.10576542648857613, 0.07297825342586531, 0.050355070176595663, 0.03474505038772526, 0.023974120624037296, 0.016542167971612474, 0.011414112971746592, 0.007875749729743213, 0.005434275441209239, 0.0037496556625463934, 0.0025872662767600914, 0.0017852164009945818, 0.0012318011590097989, 0.0008499440709219034, 0.0005864622860689734, 0.00040465958261017026, 0.00027921552960522136, 0.0001926590035749344, 0.00013293491128866099, 9.17252259765311e-05, 6.329050058322302e-05, 4.3670510717519044e-05, 3.0132697462571237e-05, 2.0791592345784034e-05, 1.4346220175283925e-05, 9.898906726086216e-06, 6.830255856561613e-06, 4.712883589775955e-06, 3.2518945405891413e-06, 2.2438105889257357e-06, 1.5482316219465221e-06, 1.068281416901081e-06, 7.371152801164351e-07, 5.08610303975247e-07, 3.509416346232248e-07, 2.421500900580104e-07, 1.670838120363444e-07, 1.1528800273382801e-07, 7.954